In [118]:
import numpy as np
import time
import copy
from __future__ import division
import traceback

# with dense vector negative sample 0, postive sample 1
def performance(f):
    def fn(*args, **kw):
        t_start = time.time()
        r = f(*args, **kw)
        t_end = time.time()
        print ('call %s() in %fs' % (f.__name__, (t_end - t_start)))
        return r
    return fn


# U is 2d-array with 2m*d dimension for each person item tuple of (x, label)

def mlr(W, U, x):
    """
    calculate mixture logistic regression
    :param U: m * d
    :param W: m * d
    :param x: d
    :return:
    """
    ux = np.dot(U, x)
    eux = softmax(ux)
    del ux
    return np.dot(eux, sigmoid(np.dot(W, x)))

def sigmoid(z):
    """
    calculate sigmoid
    :param z:
    :return:
    """
    z = z.clip(-10, 10)
    return 1 / (1 + np.exp(-z))

def softmax(x):
    """
    softmax a array
    :param x:
    :return:
    """
    e_x = np.exp(x)
    return e_x / e_x.sum()

def calLoss(X, y, weight_W, weight_U, norm21, norm1):
    """
        计算loss
    :param data:
    :param weight_W:
    :param weight_U:
    :return:
    """
    #混合逻辑回归的loss
    functionLoss = calFunctionLoss(weight_W, weight_U, X, y)
    #L21正则的loss
    norm21Loss = calNorm21(weight_W + weight_U)
    #L1正则的loss
    norm1Loss = calNorm1(weight_W + weight_U)
    print(functionLoss , norm21 * norm21Loss , norm1 * norm1Loss)
    return functionLoss + norm21 * norm21Loss + norm1 * norm1Loss

def calFunctionLossOne(W_w, W_u,x, y):
    p = mlr(W_w, W_u, x)
    if y == 0:
        return - np.log(1 - p)
    else:
        return - np.log(p)


def calFunctionLoss(W_w, W_u, X, y):
    """
    calculate the loss over all data
    :param w_w:
    :param w_u:
    :param data:
    :return:
    """
    loss = map(lambda (x,y): calFunctionLossOne(W_w,W_u,x, y), zip(X, y))
    loss = sum(loss)
    return loss
    # print("loss is:  %s" % loss)

def calNorm21(weight):
    '''
        计算norm21
    :param weight:
    :return:
    '''
    return (weight ** 2).sum() ** 0.5

def calNorm1(weight):
    """
        计算norm1
    :param weight:
    :return:
    """
    return np.abs(weight).sum()

def calDimension21(W):
    """
        计算每一个维度的L2
    :param W:
    :return:{dimension1:std1, dimension2:std2 ......}
    """
    return (W**2).sum(axis = 0) ** 0.5

# derivative for each sample
def cal_derivative(W_w, W_u, x, y):
    """
    calculate derivative
    :param weight:
    :return:
    """
    ux = np.dot(W_u, x)
    eux = softmax(ux)
    del ux
    sig = sigmoid(np.dot(W_w, x))
    mlr = np.dot(eux, sig)
    prob_scalar =  - (y - mlr) / (mlr * (1 - mlr))
    dir_U = np.outer(prob_scalar * eux * (sig - mlr), x)
    dir_W = np.outer(prob_scalar * sig * (1 - sig) * eux, x)
    return dir_W, dir_U


def sumCalDerivative(WW, WU, X, y):
    all = map(lambda (x,y): cal_derivative(WW, WU,x, y), zip(X,y))
    LW, LU = reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]),all,(0,0))
    return LW, LU




def virtualGradient(WW, WU, GW, GU,beta,lamb):
    """
    :param weight_W:
    :param weight_U:
    :param gradient_W:
    :param gradient_U:
    :param norm21:
    :param norm1:
    :return:
    """
    #计算θ_i·
    D21 = calDimension21(WW + WU)
    #计算v：
    VW = calV(GW, beta)
    VU = calV(GU, beta)
    #计算v_i·
    VD21 = calDimension21(VW + VU)
    sumVD21 = sum(VD21)
    #计算d_ij
    DW = calDij(GW, WW, VW, D21, sumVD21, beta, lamb)
    DU = calDij(GU, WU, VU, D21, sumVD21, beta, lamb)
    return DW, DU


def calV(L, beta):
    """
        计算v，包括wv， uv，这里是分别计算的
        （可以和到一起算，因为w，u一直都是分着算的，所以这里也分着算了。重构的时候再优化吧）
    :param LW:
    :param LU:
    :param beta:
    :param lamb:
    :return:
    """
    V = np.copy(L)
    V = np.maximum(np.abs(V) - beta, 0)
    return V*np.sign(-L)

def calDij(L, W, V, D21, sumVD21, beta, lamb):
    mask1 = (W != 0)
    mask2 = (W == 0) * np.tile((D21 != 0), (len(W),1))
    mask3 = np.tile((D21 == 0), (len(W),1))
    D21_tmp = np.copy(D21)
    D21_tmp[D21_tmp == 0] = 1
    s = - L - lamb * W / D21_tmp
    cond1 =  s - beta * np.sign(W)
    cond2 = np.maximum(np.abs(s) - beta, 0)*np.sign(s)
    cond3 = V * max(sumVD21 - lamb, 0) / sumVD21
    return mask1 * cond1 + mask2 * cond2 + mask3 * cond3


def loop(length, latest, direction):
    count = 0
    if(latest >= length):
        raise Exception("start should be less than length")
    if(direction == 'right'):
        while(count < length):
            if(latest + count + 1 < length):
                yield latest + count + 1
                count += 1
            else:
                yield latest + count + 1 - length
                count += 1
    elif(direction == 'left'):
        while(count < length):
            if(latest - count >= 0):
                yield latest - count
                count += 1
            else:
                yield length + latest - count
                count += 1
    else:
        raise Exception("please enter left or right")




## weight_w, weight_u, s
def lbfgs(VW, VU, sList_w,sList_u, yList_w, yList_u, k, m, start):
    """
        两个循环计算下降方向,拟合Hessian矩阵的 逆H 和梯度负方向的乘积，即 -H * f'
    :param feaNum:
    :param gk : matrix, 2m*d
    :param sList:3d*matrix,steps * 2m * d
    :param yList:3d*matrix, steps * 2m * d
    :return:
    """
    if((sList_w[start] * yList_w[start] + sList_u[start] * yList_u[start]).sum() > 0 ):
        q_u = np.copy(VU)
        q_w = np.copy(VW)
        # for delta
        L = k if k <= m else m
        alphaList = np.zeros(L)
        roList = np.zeros(L)

        for i in loop(L, start, 'left'):
            ro = 1 / (yList_w[i] * sList_w[i] + yList_u[i] * sList_u[i]).sum()
            alpha = ro * (sList_u[i] * q_u + sList_w[i] * q_w).sum()
            q_u = q_u - alpha * yList_u[i]
            q_x = q_x - alpha * yList_x[i]
            alphaList[i] = alpha
            roList[i] = ro

        for i in loop(L,start,'right'):
            ro = roList[i]
            beta = ro*(yList_u[i] * q_u + yList_w[i] * q_w).sum()
            q_u = q_u + (alphaList[i] - beta) * sList_u[i]
            q_w = q_w + (alphaList[i] - beta) * sList_w[i]

        mask_u = np.sign(q_u) * np.sign(VU) > 0
        mask_w = np.sign(q_w) * np.sign(VW) > 0

        return q_w * mask_w, q_u * mask_u
    else:
        return VW, VU

def backTrackingLineSearch(X, y, weight_W, weight_U,norm21, norm1, pW, pU):
    """
        线性搜索，得到最佳步长并更新权重
    :param it:
    :param oldLoss:
    :param data:
    :param WW:
    :param WU:
    :param GW:
    :param GU:
    :param vGW:
    :param vGU:
    :return:
    """
    alpha = 1.0
    c = 0.5
    tao = 0.9
    LW, LU = sumCalDerivative(weight_W, weight_U, X, y)
    m = (pW * LW + pU * LU).sum()
    t = - c * m
    loss = calLoss(X, y, weight_W, weight_U, norm21, norm1)

    while True:
        newW = weight_W - alpha*pW
        newU = weight_U - alpha*pU

        new_loss = calLoss(X, y, newW, newU, norm21, norm1)

        if(loss > new_loss + alpha * t):
            return newW, newU
        else:
            alpha = tao * alpha

def fixOrthant(GW, weight_W, new_weight_W):
    mask = (weight_W == 0) * np.sign(GW) + (weight_W != 0) * np.sign(weight_W)
    mask = mask * new_weight_W > 0
    return new_weight_W * mask


In [137]:
import numpy as np
import copy
import pickle
import time
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve,roc_auc_score
import random
import logging

logging.basicConfig(level=logging.DEBUG,
                format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
                datefmt='%a, %d %b %Y %H:%M:%S',
                filename='myapp.log',
                filemode='w')

class LSPLM:

    def __init__(self,
                 pieceNum = 12,
                 iterNum = 100,
                 intercept = True,
                 memoryNum = 10,
                 beta = 0.1,
                 lamb = 0.1,
                 terminate = True
                 ):
        """
        :param feaNum:  特征数
        :param classNum:    类别数
        :param iterNum:
        :param intercept:
        :param memoryNum:
        :param beta:
        :param lamb:
        :param u_stdev:
        :param w_stdev:
        """
        self.pieceNum = pieceNum
        self.iterNum = iterNum
        self.intercept = intercept
        self.memoryNum = memoryNum
        self.beta = beta
        self.lamb = lamb
        self.N = 0
        self.p = 0
        self._predict = np.vectorize(self._mlr)
        self.terminate = terminate

    def fit(self,X, y):
        """
            训练ls-plm large scale piece-wise linear model
        :param data:
        :return:
        """

        # np.random.seed(0)
        N, p = X.shape
        self.N = N
        if self.intercept:
            self.p = p + 1
            pad = np.ones((N, p + 1))
            pad[:,:-1] = X
            X = pad
            del pad
        else:
            self.p = p

        it = 0
        ## Intialization
        weight_W = np.random.normal(0,0.1,(self.pieceNum, self.p))
        weight_U = np.random.normal(0,0.1,(self.pieceNum, self.p))
        sList_w = np.zeros((self.memoryNum, self.pieceNum, self.p))
        sList_u = np.zeros((self.memoryNum, self.pieceNum, self.p))
        yList_w = np.zeros((self.memoryNum, self.pieceNum, self.p))
        yList_u = np.zeros((self.memoryNum, self.pieceNum, self.p))
        loss_before = calLoss(X, y, weight_W, weight_U, self.lamb, self.beta)
        GW, GU = sumCalDerivative(weight_W, weight_U, X, y)
        LW, LU = virtualGradient(weight_W, weight_U, GW, GU,self.beta,self.lamb)
        del GW,GU
        # print("loss: %s" % loss)
        # print("gradient_w: is")
        # for w in weight_W:
        #     print (w)
        # print("gradient_u: is")
        # for u in weight_U:
        #     print(u)
        # self.firstLoss = loss
        # self.lossList.append(loss)
        #
        print(loss_before)
        while it < self.iterNum:
            logging.info('===========iterator : %s =============' % it)
            logging.info('===========loss : %s ==============' % loss_before)
            print "iter:%d, lossdd:%s" % (it, loss_before)
            start_time = time.time()
            # 1. 计算虚梯度
                #计算梯度
            GW, GU = sumCalDerivative(weight_W, weight_U, X, y)
            print("............" + str(GW[0][0]))
            newLW, newLU = virtualGradient(weight_W, weight_U, GW, GU,self.beta,self.lamb)
            
            print(newLW[0][0])
            del GW,GU


            # dirW = copy.deepcopy(vGW)
            # dirU = copy.deepcopy(vGU)

            # 3. 利用LBFGS算法的两个循环计算下降方向, 这里会直接修改vGradient, 并确定下降方向是否跨象限

            PW, PU = lbfgs(newLW ,newLU, sList_w,sList_u, yList_w, yList_u, it, self.memoryNum, it % self.memoryNum)

            # # 4. 确定下降方向是否跨象限， 这里也会直接修改vGradient
            # fc.fixDirection(vG, dir)

            # 5. 线性搜索最优解
            # new_weight_W, new_weight_U = backTrackingLineSearch(X, y, weight_W, weight_U,self.lamb, self.beta, PW, PU)
            new_weight_W, new_weight_U = weight_W + PW, weight_U + PU

            del PW, PU

            new_weight_W = fixOrthant(newLW, weight_W, new_weight_W)
            new_weight_U = fixOrthant(newLU, weight_U, new_weight_U)
            loss_now = calLoss(X, y, new_weight_W, new_weight_U, self.lamb, self.beta)


                # 6. 判断是否提前终止
            if self.terminate and self.check(loss_before, loss_now):
                break
            else:
                overwrite = (it + 1) % self.memoryNum
                yList_u[overwrite] = LU - newLU
                yList_w[overwrite] = LW - newLW
                sList_u[overwrite] = new_weight_U - weight_U
                sList_w[overwrite] = new_weight_W - weight_W
                weight_U = new_weight_U
                weight_W = new_weight_W
                LW = newLW
                LU = newLU
                del newLW
                del newLU
                del new_weight_U
                del new_weight_W
                loss_before = loss_now

            it += 1


        logging.info("============iterator : %s end ==========" % it)
        print("")

        print("use time: ", time.time() - start_time)
        print("------------------------------------------------------\n")



        # with open("save/result"+self.stamp,"a") as fw:
        #     fw.write("train_acc:" + " ".join(ACC)+"\n")
        #     fw.write("train_loss:" + " ".join(LOSS) + "\n")
        #     fw.write("train_auc:" + " ".join(AUC) + "\n")
        #     fw.write("test_acc:" + " ".join(TEST_ACC) + "\n")
        #     fw.write("test_loss:" + " ".join(TEST_LOSS) + "\n")
        #     fw.write("test_auc:" + " ".join(TEST_AUC) + "\n")
        self.weight_W = weight_W
        self.weight_U = weight_U
    
    def check(self, loss_before, loss_now):
        return abs(loss_before - loss_now) / loss_now < 0.01

    def predict_proba(self, X):
        return self._predict(X)

    def _mlr(self, x):
        return mlr(self.weight_W,self.weight_U, (x, 1))

    def predict(self, X):
        return np.array(self._predict(X) > 0.5, dtype = int)

In [82]:
X1 = np.random.normal(1,0.2, (100, 10))
X2 = np.random.normal(-1,0.2, (100,10))

In [83]:
Y = np.zeros(200)

In [84]:
for i in range(0,100):
    Y[i] = 1

In [87]:
X = np.vstack((X1,X2))

In [88]:
X.shape

(200, 10)

In [138]:
ls = LSPLM()

In [139]:
ls.fit(X,Y)

(138.99187425734931, 0.16881353702723154, 1.5287392694446333)
140.689427064
iter:0, lossdd:140.689427064
............-11.3393598371
11.4403156844
(0.0090797798433644421, 6.2149230534891906, 50.343789580242202)
iter:1, lossdd:56.5677924136
............-2.87087148488e-05
nan
(nan, nan, nan)
iter:2, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:3, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:4, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:5, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:6, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:7, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:8, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:9, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:10, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:11, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:12, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:13, lossdd:nan
............nan


/home/chris/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:198: RuntimeWarning: invalid value encountered in true_divide
/home/chris/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:302: RuntimeWarning: invalid value encountered in greater


nan
(nan, nan, nan)
iter:14, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:15, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:16, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:17, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:18, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:19, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:20, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:21, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:22, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:23, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:24, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:25, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:26, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:27, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:28, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:29, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:30, lossdd:nan
............nan
nan
(nan, nan, nan)
iter:31, lossdd:nan
........

array([[ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan]])